In [1]:
from osgeo import gdal,ogr
import os

from datetime import datetime as dt

In [2]:
datapath = 'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiometric_Correction'
mosaicPath = 'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Mosaic_clip\\'



if not os.path.exists(mosaicPath):
        os.makedirs(mosaicPath)

shapefile = 'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Shapefile\\New_Shapefile.shp'
harmonizedFile = {}
udmFile = {}
udmFileCount = 0
harmonizedFileCount = 0

for root, dirs, files in os.walk(datapath):
    for file in files:
        if file.endswith('_harmonized.tif'):
            date = file.split('_')[0]
            if date not in harmonizedFile:
                harmonizedFile[date] = root + '/' + file
                
            else:
                harmonizedFile[date] = harmonizedFile[date] + ',' +root + '/' + file
            harmonizedFileCount += 1
        if file.endswith('_udm2.tif'):
            date = file.split('_')[0]
            if date not in udmFile:
                udmFile[date] = root + '/' + file
                
            else:
                udmFile[date] = udmFile[date] + ',' +root + '/' + file
            udmFileCount += 1
            
print("Total Harmonized Files: " + str(harmonizedFileCount))

print("Total UDM Files: " + str(udmFileCount))

print('-------------------')
print("Harmonized Files: " + str(harmonizedFile))

print("UDM Files: " + str(udmFile))


Total Harmonized Files: 554
Total UDM Files: 554
-------------------
Harmonized Files: {'20201011': 'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiometric_Correction\\2020\\10. Oct/20201011_043212_88_2274_3B_AnalyticMS_SR_8b_harmonized.tif', '20201015': 'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiometric_Correction\\2020\\10. Oct/20201015_034121_68_2257_3B_AnalyticMS_SR_8b_harmonized.tif,E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiometric_Correction\\2020\\10. Oct/20201015_043056_68_2416_3B_AnalyticMS_SR_8b_harmonized.tif,E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiometric_Correction\\2020\\10. Oct/20201015_043059_03_2416_3B_AnalyticMS_SR_8b_harmonized.tif', '20201107': 'E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiom

In [3]:
# mosaic

In [4]:
for i in harmonizedFile.keys():
    print(i)
    if len(harmonizedFile[i].split(",")) < 2:
        print("Only one file for " + i)
        
    elif len(harmonizedFile[i].split(",")) > 2:
        print(f'Count on {i} is {len(harmonizedFile[i].split(","))}')
    print('-'*20)

20201011
Only one file for 20201011
--------------------
20201015
Count on 20201015 is 3
--------------------
20201107
--------------------
20201109
Only one file for 20201109
--------------------
20201128
Only one file for 20201128
--------------------
20201129
--------------------
20201227
--------------------
20201228
--------------------
20201231
Only one file for 20201231
--------------------
20200327
--------------------
20200430
--------------------
20200515
--------------------
20210110
--------------------
20211001
--------------------
20211004
--------------------
20211010
Only one file for 20211010
--------------------
20211014
--------------------
20211029
Count on 20211029 is 3
--------------------
20211111
Count on 20211111 is 4
--------------------
20211115
Count on 20211115 is 3
--------------------
20211119
--------------------
20211123
Only one file for 20211123
--------------------
20211126
--------------------
20211129
--------------------
20211211
-----------------

In [5]:
# mosaic
for date in harmonizedFile.keys():
    filestoMosaic = harmonizedFile[date].split(',')
    date = dt.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
    date = str(date)
    if len(filestoMosaic) < 2:
        print("Only one file for " + date)
        
    # print(filestoMosaic)
    outvrt = mosaicPath +"/"+ date + '_mosaic.vrt'
    
    outtif = mosaicPath +"/"+ date + '_mosaic.tif'
    
    if (date+'_mosaic_clip.tif') in os.listdir(mosaicPath):
        print("Already Mosaiced " + outtif)
        continue

    gdal.BuildVRT(outvrt, filestoMosaic)
    gdal.Translate(outtif, outvrt)
    print("Mosaiced " + outtif)
    
    # rename bands
    
    ds = gdal.Open(outtif,gdal.GA_Update)
    ds.GetRasterBand(1).SetDescription('Coastal Blue')
    ds.GetRasterBand(2).SetDescription('Blue')
    ds.GetRasterBand(3).SetDescription('Green I')
    ds.GetRasterBand(4).SetDescription('Green')
    ds.GetRasterBand(5).SetDescription('Yellow')
    ds.GetRasterBand(6).SetDescription('Red')
    ds.GetRasterBand(7).SetDescription('Red Edge')
    ds.GetRasterBand(8).SetDescription('Near-infrared')
    ds = None
    print("Renamed Bands for " + outtif)
   
    
    # delete vrt
    os.remove(outvrt)
    print("Deleted " + outvrt)
    
    #clip
    outClip = mosaicPath+'/' + date + '_mosaic_clip.tif'
    gdal.Warp(outClip, outtif, cutlineDSName=shapefile, cropToCutline=True)
    print("Clipped " + outClip)
    
    # delete mosaic
    os.remove(outtif)
    print("Deleted " + outtif)
    
    #remove files to mosaic 
    for i in filestoMosaic:
        os.remove(i)
        print("Deleted " + i)
    print('-'*20)


Only one file for 2020-10-11
Mosaiced E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_mosaic.tif
Renamed Bands for E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_mosaic.tif
Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_mosaic.vrt
Clipped E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_mosaic_clip.tif
Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_mosaic.tif
Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Radiometric_Correction\2020\10. Oct/20201011_043212_88_2274_3B_AnalyticMS_SR_8b_harmonized.tif
--------------------
Mosaiced E:\LULC Change Detection Using Unsupervised Clustering 

In [6]:
gdal.GetLastErrorMsg()


''

In [7]:
print(udmFile.keys())


dict_keys(['20201011', '20201015', '20201107', '20201109', '20201128', '20201129', '20201227', '20201228', '20201231', '20200327', '20200430', '20200515', '20210110', '20211001', '20211004', '20211010', '20211014', '20211029', '20211111', '20211115', '20211119', '20211123', '20211126', '20211129', '20211211', '20211215', '20211226', '20211230', '20210205', '20210211', '20210302', '20210317', '20210414', '20210516', '20210517', '20210714', '20210904', '20220104', '20220105', '20220112', '20220117', '20220121', '20220122', '20220127', '20220131', '20221009', '20221010', '20221015', '20221020', '20221026', '20221027', '20221028', '20221031', '20221102', '20221105', '20221107', '20221108', '20221114', '20221116', '20221118', '20221122', '20221123', '20221124', '20221127', '20221204', '20221205', '20221207', '20221213', '20221215', '20221217', '20221220', '20221222', '20221231', '20220201', '20220203', '20220208', '20220209', '20220210', '20220213', '20220215', '20220217', '20220218', '2022

In [8]:
for i in udmFile.keys():
    print(i)
    print(f'Count on {i} is {len(udmFile[i].split(","))}')
    print('-'*20)

20201011
Count on 20201011 is 1
--------------------
20201015
Count on 20201015 is 3
--------------------
20201107
Count on 20201107 is 2
--------------------
20201109
Count on 20201109 is 1
--------------------
20201128
Count on 20201128 is 1
--------------------
20201129
Count on 20201129 is 2
--------------------
20201227
Count on 20201227 is 2
--------------------
20201228
Count on 20201228 is 2
--------------------
20201231
Count on 20201231 is 1
--------------------
20200327
Count on 20200327 is 2
--------------------
20200430
Count on 20200430 is 2
--------------------
20200515
Count on 20200515 is 2
--------------------
20210110
Count on 20210110 is 2
--------------------
20211001
Count on 20211001 is 2
--------------------
20211004
Count on 20211004 is 2
--------------------
20211010
Count on 20211010 is 1
--------------------
20211014
Count on 20211014 is 2
--------------------
20211029
Count on 20211029 is 3
--------------------
20211111
Count on 20211111 is 4
--------------

In [10]:
# mosaic udm    
for date in udmFile.keys():
    filestoMosaic = udmFile[date].split(',')

    date = dt.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
    date = str(date)
    print(filestoMosaic)
    outvrt = mosaicPath + "/"+ date + '_udm_mosaic.vrt'
    outtif = mosaicPath +"/" +date + '_udm_mosaic.tif'
    if (date+'_udm_mosaic_clip.tif') in os.listdir(mosaicPath):
        print("Already Mosaiced " + outtif)
        continue
        
    gdal.BuildVRT(outvrt, filestoMosaic, )
    gdal.Translate(outtif, outvrt)
    print("Mosaiced " + outtif)
    # rename bands
    ds = gdal.Open(outtif, gdal.GA_Update)
    ds.GetRasterBand(1).SetDescription('Clear')
    ds.GetRasterBand(2).SetDescription('Cloud')
    ds.GetRasterBand(3).SetDescription('Cloud Shadow')
    ds.GetRasterBand(4).SetDescription('Light Haze')
    ds.GetRasterBand(5).SetDescription('Heavy Haze')
    ds.GetRasterBand(6).SetDescription('Snow')
    ds.GetRasterBand(7).SetDescription('Confidence')
    ds.GetRasterBand(8).SetDescription('Unusable Pixels')
    ds = None
    print("Renamed Bands for " + outtif)
    # delete vrt
    os.remove(outvrt)
    print("Deleted " + outvrt)
    
    #clip
    outClip = mosaicPath +"/"+ date + '_udm_mosaic_clip.tif'    
    gdal.Warp(outClip, outtif, cutlineDSName=shapefile, cropToCutline=True)
    print("Clipped " + outClip)
    
    # delete mosaic
    os.remove(outtif)
    print("Deleted " + outtif)
    
    for i in filestoMosaic:
        os.remove(i)
        print("Deleted " + i)
    print('-'*20)


['E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiometric_Correction\\2020\\10. Oct/20201011_043212_88_2274_3B_udm2.tif']
Mosaiced E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_udm_mosaic.tif
Renamed Bands for E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_udm_mosaic.tif
Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_udm_mosaic.vrt
Clipped E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_udm_mosaic_clip.tif
Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Mosaic_clip\/2020-10-11_udm_mosaic.tif
Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Radiometric_Corr

In [11]:
gdal.GetLastErrorMsg()


''